# Random forest 

The steps: 
1. Load the merged_df dataset. This has 111 rows. 
2. Split it into train and test. Test should be 0.3, this leads to better recall. 
3. Apply SMOTE on the train set. This creates X_train_res (res stands for resampled) and y_train_res. 
4. 

Import libraries. 

In [203]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import confusion_matrix as cm
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

Merged df is the dataframe I created with all the extracted features from TS fresh. 

In [204]:
merged_df = pd.read_csv('/Users/dionnespaltman/Desktop/V3/merged_df.csv', sep=',')

merged_df.drop('Unnamed: 0', axis=1, inplace=True)
merged_df.drop('Unnamed: 0.1', axis=1, inplace=True)

display(merged_df.head(5))

,ID,sum_12,sum_4567,sum_456,VVR_group,Condition,VVR_1,VVR_2,AU01_r__sum_values,AU01_r__variance,...,AU26_r__minimum,AU26_r__mean,AU26_r__mean_abs_change,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__minimum,AU45_r__mean,AU45_r__mean_abs_change
0,23,24.0,37.0,27.0,0,2,13.0,11.0,4982.48,0.425041,...,0.00,0.633284,0.076328,9231.74,0.825039,0.908316,4.91,0.00,0.627753,0.133624
1,24,23.0,37.0,28.0,0,2,12.0,11.0,9390.23,0.448366,...,0.00,1.484701,0.125851,11887.00,0.634554,0.796589,5.00,0.00,0.436942,0.098134
2,25,28.0,44.0,33.0,1,2,16.0,12.0,6954.35,0.599805,...,0.00,0.862301,0.101969,9020.78,0.750701,0.866430,4.04,0.00,0.550652,0.085720
3,26,30.0,37.0,29.0,0,1,15.0,15.0,9707.43,0.873280,...,0.00,0.552359,0.069582,6585.31,0.609348,0.780607,4.90,0.00,0.371673,0.056287
4,27,22.0,39.0,31.0,1,2,11.0,11.0,21049.90,1.475421,...,-3.92,0.142027,0.386527,23027.73,1.160635,1.077328,5.04,-4.29,1.094318,0.231853


I want to know how many people are in each group. 

In [205]:
# Count the number of instances of people in VVR_group = 1 and VVR_group = 0
count_vvr_group = merged_df['VVR_group'].value_counts()

# Print the counts
print("Number of instances in VVR_group = 1:", count_vvr_group[1])
print("Number of instances in VVR_group = 0:", count_vvr_group[0])

Number of instances in VVR_group = 1: 26
Number of instances in VVR_group = 0: 85


In [ ]:
columns_to_drop = [ 'ID', 'sum_12', 'sum_4567', 'sum_456', 'VVR_group', 'Condition'] 

In [210]:
X = merged_df.drop(columns_to_drop, axis=1)
y = merged_df['VVR_group']

These are the columns I use to predict, so all my features. I need these as a list to establish my featurizer. 
I have 119 features from TS fresh and then I added the two VVR measurements from stage 1 and 2. 

In [206]:
with open('/Users/dionnespaltman/Desktop/V3/columns_au_12.json', 'r') as f:
    columns_au_12 = json.load(f)

print(len(columns_au_12))
# print(columns_au_12)

121


First we'll split the data into a train and test set. 

The train set has 88 participants, the test set has 23 participants. 

I have made the test_size 30% instead of 20%. 
Then I tested test_size 40% and that made the recall slightly higher (but not by much). 

In [208]:
train, test = train_test_split(merged_df, test_size=0.4, random_state=123, stratify=merged_df['VVR_group'])
# train, val = train_test_split(train, stratify=train['VVR_group'], random_state=123)

print(train.shape)
print(test.shape)

(66, 127)
(45, 127)


Unfortunately, the test set is very small with only 8 people in the high VVR condition. 

In [209]:
columns_to_drop = [ 'ID', 'sum_12', 'sum_4567', 'sum_456', 'VVR_group', 'Condition'] 

X_test = test.drop(columns_to_drop, axis=1)
y_test = test['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_test))

Original dataset shape Counter({0: 34, 1: 11})


# Applying SMOTE

In [187]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

In [188]:
columns_to_drop = [ 'ID', 'sum_12', 'sum_4567', 'sum_456', 'VVR_group', 'Condition'] 

X_train = train.drop(columns_to_drop, axis=1)
y_train = train['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_train))

# Apply SMOTE to the training data with sampling strategy set to 'auto' (default)
sm = SMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=5)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Print resampled class distribution
print('Resampled dataset shape %s' % Counter(y_train_res))

Original dataset shape Counter({0: 51, 1: 15})
Resampled dataset shape Counter({1: 51, 0: 51})


# Adding class weights

I will add class weights to my models, because of my inbalanced data set. 
https://medium.com/@ravi.abhinav4/improving-class-imbalance-with-class-weights-in-machine-learning-af072fdd4aa4 

In [189]:
import numpy as np

def calculate_class_weights(y):
    unique_classes, class_counts = np.unique(y, return_counts=True)
    total_samples = len(y)
    class_weights = {}

    for class_label, class_count in zip(unique_classes, class_counts):
        class_weight = total_samples / (2.0 * class_count)
        class_weights[class_label] = class_weight

    return class_weights

# Assuming 'y' contains the class labels (0s and 1s) for the binary classification problem
class_weights = calculate_class_weights(y_train)
print("Class weights:", class_weights)

# this is how you would implement it 
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], class_weight=class_weights)

Class weights: {0: 0.6470588235294118, 1: 2.2}


In [190]:
featurizer = ColumnTransformer(transformers=[("numeric", StandardScaler(), columns_au_12)], remainder='drop')

# Model 

Strange enough the recall is much higher (0.27 instead of 0.18) when I don't use class weights. 

In [211]:
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

# model = make_pipeline(featurizer, RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0, class_weight=class_weights))
model = make_pipeline(featurizer, RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0))

model.fit(X_train_res, y_train_res)

print('Original dataset shape %s' % Counter(y_train_res))

pred = model.predict(test.drop('VVR_group', axis=1))
accuracy = accuracy_score(test['VVR_group'].values, pred)
report = classification_report(test['VVR_group'].values, pred)
cm = confusion_matrix(test['VVR_group'].values, pred)
precision, recall, _ = precision_recall_curve(test['VVR_group'].values, pred)
auc_pr = auc(recall, precision)

# Print results
print(f"Accuracy on Validation Data: {accuracy}")
print(f"AUC-PR on Validation Data: {auc_pr}")
print("Classification Report:")
print(report)

Original dataset shape Counter({1: 51, 0: 51})
Accuracy on Validation Data: 0.7777777777777778
AUC-PR on Validation Data: 0.5252525252525253
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.94      0.86        34
           1       0.60      0.27      0.38        11

    accuracy                           0.78        45
   macro avg       0.70      0.61      0.62        45
weighted avg       0.75      0.78      0.75        45



# Recursive feature elimination to optimize the scores 
https://machinelearningmastery.com/rfe-feature-selection-in-python/

In [214]:
print(X.shape)
print(y.shape)

(111, 121)
(111,)


Recall is down below. 

In [219]:
# evaluate RFE for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='recall', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Recall: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Recall: 0.533 (0.306)


Precision. 

In [221]:
# evaluate RFE for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='precision', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Precision: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Precision: 0.458 (0.302)


/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy. 

In [223]:
# evaluate RFE for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.736 (0.121)


Recall per class. 

KeyError: "None of [Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,\n             13,  14,  15,  16,  17,  18,  19,  22,  24,  25,  26,  27,  28,\n             29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,\n             42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  55,\n             56,  57,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,\n             71,  73,  74,  75,  76,  77,  79,  80,  81,  82,  83,  84,  85,\n             86,  87,  88,  89,  90,  91,  93,  94,  96,  97,  99, 100, 101,\n            103, 104, 105, 106, 107, 108, 109, 110],\n           dtype='int64')] are in the [columns]"